# Are We Rich Yet? HW4 version

In [1]:
%matplotlib inline
import datetime
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("poster")

## Q1 EDA and the Baseline Model

The data file here that you read in is a database of reviews, one review per row from the Yelp web site.  The columns are:

* `user_id`: an opaque id for the reviewer
* `business_id`: an opaque id for the restaurant
* `date`: the date the review was made
* `review_id`: on opaque id for the review
* `stars`: the number of stars the user gave it
* `text`: the text of the review
* `business_avg`: the average rating this business got
* `business_review_count`: the number of reviews this business had
* `user_avg`: the average rating this user gave
* `user_review_count`: the number of reviews this user did.

Users rate restaurants on a scale of 1-5. Even though this rating is integer valued, for the purposes of this assignment we shall treat it as a real number.

We start with a 31MB in size data set and create a smaller one to start. The 31MB dataset is already too large for most of the computations in this notebook to complete within a few minutes, so we will need to pare it down.

In [2]:
df=pd.read_csv("data/IYR.csv")
df.date = df['date'].apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d'))
dftouse = df.copy()
dftouse.tail()

,date,open,high,low,close,volume,close_1,result_1,perf_1,close_14,result_14,perf_14,results,bb_upper,bb_middle,bb_lower,bb_pct,bb_bandwidth,bb_squeeze,bb_signalup,bb_signaldn,bb_signal,ema50,ema150,ema200,ema_signal1,ema_signal2,kama50,kama150,kama200,kama_signal1,kama_signal2,sar,sar_signal,adx,plus_di,minus_di,adx_trend,adx_direction,adx_signal,aroon_osc,aroon_signal,cci,cci_signal,macd,macd_sigline,macd_hist,macd_signal,ppo,ppo_signal,mfi,mfi_signal,roc,roc_signal,rsi,rsi_signal,ult_osc,ult_signal,willr,wr_signal,ad_osc,ad_signal,stoch_slowk,stoch_slowd,sslow_signal,stoch_fastk,stoch_fastd,srsi_signal,trix,trix_signal,sr_pivotpts,sr_res1,sr_sup1,sr_res2,sr_sup2,sr_res3,sr_sup3,cv_signal
1478,2015-11-16,72.300003,73.309998,72.300003,73.290001,5939900,73.330002,True,0.000546,61.07,False,-0.166735,1,77.820998,75.097000,72.373003,0.168318,7.254610,False,False,False,1,73.955311,74.243210,74.345325,0,0,73.759056,74.300903,75.396076,0,0,76.061274,0,26.601219,13.698139,29.176343,True,False,0,-40,0,-88.983026,1,-0.212274,0.235812,-0.448086,0,-0.639207,0,42.565106,0,-4.830543,1,44.321554,1,42.429896,0,-79.113957,1,-2735546.954670,0,27.219841,25.850562,1,86.823377,42.999771,0,0.058669,1,72.966667,73.633332,68.893334,77.706665,68.226669,78.373330,64.153336,0
1479,2015-11-17,73.190002,73.959999,72.959999,73.330002,9096300,74.110001,True,0.010637,59.48,False,-0.188872,1,77.753154,74.961500,72.169847,0.207790,7.448232,False,False,False,1,73.930789,74.231115,74.335222,0,0,73.750932,74.291788,75.372379,0,0,75.760373,0,26.317597,17.164157,27.205156,True,False,0,-40,0,-57.559205,1,-0.254883,0.137673,-0.392556,0,-0.800961,0,39.684043,0,-3.905119,1,44.580679,1,45.184818,0,-78.270054,1,-3147116.232380,0,38.029741,28.505516,1,90.799039,59.207472,0,0.056971,1,73.416667,74.533330,69.793332,78.156665,68.676669,79.273328,65.053334,0
1480,2015-11-18,73.589996,74.169998,73.120003,74.110001,8293200,74.320000,True,0.002834,60.25,False,-0.187019,1,77.667774,74.878501,72.089227,0.362240,7.450132,False,False,False,1,73.937817,74.229511,74.332981,0,0,73.761425,74.290368,75.359310,0,0,75.483543,0,25.762636,17.365427,25.274308,True,False,0,-40,0,-32.375396,1,-0.223140,0.065511,-0.288650,0,-1.123941,0,39.859897,0,-2.409798,1,49.514647,1,52.027057,0,-61.814372,1,-692195.818664,0,70.091546,45.113709,1,100.000000,92.540805,0,0.055227,1,73.800001,75.299998,70.560000,78.539999,69.060003,80.039996,65.820002,0
1481,2015-11-19,74.279999,74.620003,74.019997,74.320000,6536100,75.080002,True,0.010226,59.52,False,-0.199139,1,77.447361,74.764501,72.081641,0.417159,7.176829,False,False,False,1,73.952804,74.230709,74.332852,1,0,73.778100,74.290596,75.347808,1,0,75.228860,0,24.643201,19.777373,24.216607,True,False,0,-40,0,1.377413,1,-0.178975,0.016614,-0.195588,0,-1.380651,0,45.259109,0,-2.415972,1,50.785046,1,49.923065,0,-57.384012,1,375016.697735,1,80.575192,62.898826,0,100.000000,96.933013,0,0.053529,1,74.320000,76.339997,71.599999,79.059998,69.580002,81.079995,66.860001,0
1482,2015-11-20,74.389999,75.169998,74.379997,75.080002,7804800,65.150002,False,-0.132259,59.02,False,-0.213905,0,77.361753,74.724001,72.086249,0.567482,7.059986,False,False,False,1,73.997008,74.241958,74.340286,1,0,73.814288,74.295965,75.345608,1,0,72.300003,1,22.928698,22.473685,22.763272,True,False,0,-40,0,48.016317,1,-0.081706,-0.003050,-0.078656,0,-1.506564,0,39.645890,0,1.500610,1,55.180686,1,52.274667,0,-37.807545,1,2695056.981140,1,93.574892,81.413877,0,100.000000,100.000000,0,0.052153,1,74.876666,77.453328,72.713330,79.616664,70.136668,82.193326,67.973332,0


In [3]:
IGNORE = ['date', 'result_1','close_1','perf_1','result_14','close_14','perf_14','results']

INDICATORS=[]
for v in dftouse.columns:
    l=dftouse[v].unique()
    if len(l) <= 10 and v not in IGNORE:
        #print v, l
        INDICATORS.append(v)

STANDARDIZABLE = []
for v in dftouse.columns:
    if v not in INDICATORS and v not in IGNORE:
        #print v
        STANDARDIZABLE.append(v)

lcols=[]
lcols.append('date')
for c in list(dftouse.columns):
    if c not in IGNORE:
        lcols.append(c)

#from sklearn.cross_validation import train_test_split
#itrain, itest = train_test_split(xrange(dftouse.shape[0]), train_size=0.8)

#mask=np.ones(dftouse.shape[0], dtype='int')
#mask[itrainval]=1
#mask[itest]=0
#mask = (mask==1)

mask = dftouse['date']<datetime.date(2015, 1, 1)

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(dftouse[mask][STANDARDIZABLE])
dftouse[STANDARDIZABLE] = scaler.transform(dftouse[STANDARDIZABLE])

Xmatrix=dftouse[lcols]#.values
Yresp=dftouse['results']#.values

traindf=Xmatrix[mask]
testdf=Xmatrix[~mask]
#traindf=pd.DataFrame(Xmatrix[mask], columns=lcols)
#testdf=pd.DataFrame(Xmatrix[~mask], columns=lcols)
Yresp_train=Yresp[mask]
Yresp_test=Yresp[~mask]

print traindf.shape, testdf.shape

(1258, 71) (225, 71)


In [4]:
#traindf=recompute_frame(traindf)
#validatedf=recompute_frame(validatedf)
#testdf=recompute_frame(testdf)
#validatedf=validatedf[['user_id', 'business_id','stars', 'review_id']]
#testdf=testdf[['user_id', 'business_id', 'stars', 'review_id']]
traindf.head()

,date,open,high,low,close,volume,bb_upper,bb_middle,bb_lower,bb_pct,bb_bandwidth,bb_squeeze,bb_signalup,bb_signaldn,bb_signal,ema50,ema150,ema200,ema_signal1,ema_signal2,kama50,kama150,kama200,kama_signal1,kama_signal2,sar,sar_signal,adx,plus_di,minus_di,adx_trend,adx_direction,adx_signal,aroon_osc,aroon_signal,cci,cci_signal,macd,macd_sigline,macd_hist,macd_signal,ppo,ppo_signal,mfi,mfi_signal,roc,roc_signal,rsi,rsi_signal,ult_osc,ult_signal,willr,wr_signal,ad_osc,ad_signal,stoch_slowk,stoch_slowd,sslow_signal,stoch_fastk,stoch_fastd,srsi_signal,trix,trix_signal,sr_pivotpts,sr_res1,sr_sup1,sr_res2,sr_sup2,sr_res3,sr_sup3,cv_signal
0,2010-01-04,-1.974789,-2.021873,-2.010345,-2.057763,1.257904,-2.150531,-2.061784,-1.942517,-0.223879,0.305631,False,False,False,0,-2.219221,-2.371982,-2.298187,1,1,-2.355040,-2.339656,-2.535415,1,1,-1.692185,0,0.578886,0.425673,-0.134153,True,True,1,0.961322,1,-0.422823,1,0.964501,1.204992,-0.391073,0,1.286500,1,0.260301,0,0.219601,0,-0.037623,0,-1.324320,0,-0.771631,0,-0.859011,0,-1.575159,-1.161855,1,-1.275851,-1.530970,1,1.620204,1,-2.031051,-1.927240,-1.791896,-2.103268,-1.826986,-1.893877,-1.599763,1
1,2010-01-05,-2.056848,-2.112559,-2.040752,-2.043666,0.807210,-2.147773,-2.058671,-1.939132,-0.154135,0.300964,False,False,False,0,-2.210674,-2.363794,-2.291755,1,1,-2.348911,-2.329997,-2.520053,1,1,-1.703985,0,0.391414,0.192819,-0.023553,True,True,1,0.961322,1,-0.737174,1,0.842441,1.147439,-0.583850,0,1.178123,1,-0.236236,0,0.061919,0,0.031994,0,-1.148810,0,-0.843764,0,-0.517596,0,-1.709058,-1.654943,1,-1.062164,-1.510250,1,1.644016,1,-2.066518,-1.999432,-1.855567,-2.141951,-1.857354,-1.968885,-1.653827,1
2,2010-01-06,-2.040180,-2.077580,-1.987539,-2.046229,0.869554,-2.148629,-2.050565,-1.922867,-0.218532,0.224415,False,False,False,0,-2.202566,-2.355748,-2.285410,1,1,-2.341953,-2.320493,-2.507433,1,1,-1.722771,0,0.257313,0.394016,-0.136481,True,True,1,0.894666,1,-0.583524,1,0.731427,1.077185,-0.713485,0,1.037635,1,0.235718,0,-0.304044,0,0.015896,0,-1.464693,0,-0.865131,0,-0.538726,0,-1.313851,-1.655338,1,-1.111575,-1.441590,1,1.660148,1,-2.037888,-1.941157,-1.804170,-2.110725,-1.832840,-1.908336,-1.610185,1
3,2010-01-07,-2.049155,-2.040010,-2.030616,-1.993686,1.148834,-2.150957,-2.038049,-1.896822,0.066723,0.098578,False,False,False,0,-2.192676,-2.347129,-2.278627,1,1,-2.332593,-2.310767,-2.492155,1,1,-1.740430,0,0.070410,-0.005315,-0.019360,True,True,1,0.828010,1,-0.510848,1,0.685206,1.010902,-0.672958,0,0.964794,1,0.194813,0,-0.352797,0,0.291625,0,-0.830668,0,-0.427009,0,-0.099395,1,-0.789409,-1.372476,1,1.104482,-0.446710,0,1.672075,1,-2.022505,-1.909845,-1.776554,-2.093947,-1.819668,-1.875803,-1.586736,1
4,2010-01-08,-2.008126,-2.077580,-2.002743,-2.033414,0.084989,-2.161656,-2.027154,-1.866116,-0.286395,-0.079749,False,False,False,0,-2.184761,-2.339137,-2.272291,1,1,-2.323487,-2.301715,-2.477993,1,1,-1.757029,0,-0.103143,-0.235088,-0.174363,True,True,1,0.761354,1,-0.765480,1,0.598393,0.938942,-0.730899,0,0.842637,1,0.284929,0,-0.560898,0,0.022547,0,-0.317122,0,-0.758274,0,0.176323,1,-0.458178,-0.922170,1,-0.840844,-0.354208,1,1.677631,1,-2.038743,-1.942896,-1.805704,-2.111657,-1.833572,-1.910144,-1.611488,1


Lets save these files out so that we can use them if we lose our original files.

In [5]:
#traindf.to_csv("tempdata/strain.csv", index=False, header=True, encoding="utf-8")
#validatedf.to_csv("tempdata/svalidate.csv", index=False, header=True, encoding="utf-8")
#testdf.to_csv("tempdata/stest.csv", index=False, header=True, encoding="utf-8")

In [6]:
testdf.head()

,date,open,high,low,close,volume,bb_upper,bb_middle,bb_lower,bb_pct,bb_bandwidth,bb_squeeze,bb_signalup,bb_signaldn,bb_signal,ema50,ema150,ema200,ema_signal1,ema_signal2,kama50,kama150,kama200,kama_signal1,kama_signal2,sar,sar_signal,adx,plus_di,minus_di,adx_trend,adx_direction,adx_signal,aroon_osc,aroon_signal,cci,cci_signal,macd,macd_sigline,macd_hist,macd_signal,ppo,ppo_signal,mfi,mfi_signal,roc,roc_signal,rsi,rsi_signal,ult_osc,ult_signal,willr,wr_signal,ad_osc,ad_signal,stoch_slowk,stoch_slowd,sslow_signal,stoch_fastk,stoch_fastd,srsi_signal,trix,trix_signal,sr_pivotpts,sr_res1,sr_sup1,sr_res2,sr_sup2,sr_res3,sr_sup3,cv_signal
1258,2015-01-02,2.002478,2.030497,1.986947,2.040620,0.501033,2.044409,2.027570,1.975538,0.171600,-0.609179,False,False,False,1,1.926164,1.690669,1.630328,1,1,1.994191,1.769933,1.759361,1,1,1.901290,1,-0.506879,-0.092337,0.295197,False,False,0,-0.038521,1,-0.046308,0,0.580460,0.765637,-0.338559,0,0.160339,1,0.791813,0,0.210315,1,0.123754,0,-1.333413,0,0.267223,0,-0.116334,1,-0.978255,-1.095054,0,0.138707,-0.695441,1,0.817325,1,2.020289,2.009805,1.892183,2.058155,1.843972,1.947305,1.708242,1
1259,2015-01-05,2.017863,2.066771,2.032558,2.073941,1.408488,2.054965,2.033990,1.978012,0.343512,-0.591174,False,False,False,1,1.936650,1.698271,1.637013,1,1,2.000466,1.776471,1.768711,1,1,1.901290,1,-0.656058,0.043484,0.117842,False,True,0,-0.038521,1,0.158832,0,0.581519,0.739050,-0.267769,0,0.290884,1,1.386404,0,0.043996,1,0.258281,0,-0.882109,0,0.470486,0,0.328092,1,-0.555328,-0.971577,0,0.492731,-0.269143,1,0.815598,1,2.058747,2.088085,1.961224,2.100100,1.876902,2.028638,1.766866,1
1260,2015-01-06,2.099922,2.158753,2.122513,2.146989,0.362173,2.075996,2.046701,1.982786,0.712898,-0.554971,False,False,False,1,1.949643,1.706716,1.644330,1,1,2.009600,1.784889,1.779535,1,1,1.924897,1,-0.696435,0.703923,-0.070813,False,True,0,-0.038521,1,0.729411,0,0.645231,0.731676,-0.071666,0,0.348997,1,1.820184,0,0.198926,1,0.541979,0,-0.202605,0,0.916097,0,0.545836,1,0.473343,-0.381752,0,1.104482,0.725736,0,0.815354,1,2.143782,2.220727,2.113880,2.149502,1.983741,2.124432,1.926778,1
1261,2015-01-07,2.153773,2.272758,2.165590,2.277706,0.149003,2.123688,2.063822,1.971276,1.259359,-0.414651,False,True,False,1,1.967349,1.716740,1.652823,1,1,2.026880,1.796949,1.795138,1,1,1.946616,1,-0.633661,1.364241,-0.326540,False,True,0,0.561385,1,1.424387,0,0.814732,0.762742,0.320710,1,0.404222,1,2.369259,0,0.096572,1,0.983829,0,0.270004,0,1.189346,0,0.889806,1,0.966744,0.318430,0,1.104482,1.129390,0,0.820052,1,2.239499,2.415556,2.207468,2.348974,1.991058,2.419037,1.939804,1
1262,2015-01-08,2.306349,2.332351,2.289753,2.335375,-0.011060,2.178643,2.082175,1.955388,1.284084,-0.246657,False,True,False,1,1.986664,1.727376,1.661782,1,1,2.046265,1.808882,1.813041,1,1,1.980059,1,-0.529930,1.634582,-0.467708,False,True,0,0.628041,1,1.622966,0,0.992442,0.826353,0.652886,1,0.456030,1,2.338109,0,0.416138,1,1.156865,0,0.217145,0,1.146013,0,1.145057,1,1.282653,0.980086,0,1.104482,1.385076,0,0.830153,1,2.320261,2.579944,2.299522,2.501375,2.009718,2.652192,1.973025,1


In [7]:
def calculate_commission(price, quantity):
    #fix rate pricing from IB including all exchange and regularotry fees
    #https://www.interactivebrokers.com/en/index.php?f=1590&p=stocks1&ns=T
    return np.minimum(np.maximum(0.005*quantity, 1), price*quantity*0.005)

'''
input:
signal_column: name of the column that indicates the signal,  1 for long signal or -1 for short signal
price_column: name of the column responding to order execution price, could be open or close
include_commision: deduct commission for each transaction if True, default to True

output: 
balance, profit, ROI

NOTE: this is LONG only evaluation, SHORT is not implemented yet. SHORT position will involve margin, so we need to decide 
later.

'''
def evaluate_profit(df, start_date, end_date, intitial_balance, signal_column, price_column, include_commission=True):
    df = df.sort('date')
    df = df[df.date>=start_date][df.date<=end_date]
    
    balance = intitial_balance
    current_position = 0 #1 for long position, -1 for short, 0 for closed position or no position
    pos_quantity = 0 # current position quantity
    pos_price = 0 #current position price

    #iterate from first to n-1 row, as we want to close our position on the last day.
    for index, row in df.head(-1).iterrows():
        new_position = row[signal_column]
        current_price = row[price_column]
        
        # receive signal to reverse current position
        if current_position != new_position:
            # bullish market: close short position and open a long position
            if new_position == 1:
                #open a long position (short position not implemented yet)
                pos_quantity = np.floor_divide(balance, current_price) 
                balance = balance - current_price * pos_quantity
                if include_commission:
                    balance = balance - calculate_commission(current_price, pos_quantity)
                
                pos_price = current_price
                current_position = new_position
                #print "long ", row['date'], balance, pos_quantity, pos_price
            
            # bearish market: close long position and open a short position
            else:
                #close the long position (short position not implemented yet)
                balance = balance + current_price * pos_quantity
                if include_commission:
                    balance = balance - calculate_commission(current_price, pos_quantity)
                    
                pos_quantity = 0
                pos_price = 0
                current_position = new_position
                #print "close", row['date'], balance, pos_quantity, pos_price
            
        #hold position and do nothing
        else:
            #print "hold ", row['date'], balance, pos_quantity, pos_price
            pass
            
            
    #found open position on the last day, let's close it
    if pos_quantity != 0:
        # get price of last day
        current_price = df.tail(1).iloc[0][price_column]
        balance = balance + pos_quantity * current_price
        if include_commission:
            balance = balance - calculate_commission(current_price, pos_quantity)
    
    profit = balance - intitial_balance
    ROI = profit/intitial_balance
    
    return  balance, profit, ROI

In [8]:
'''
Example for evaluate_profit() -- Baseline train on scaled traindf with inverse transform
'''
start_date = datetime.date(2010, 1, 1)
end_date = datetime.date(2014, 12, 31)
df_test = traindf.copy()
df_test[STANDARDIZABLE] = scaler.inverse_transform(df_test[STANDARDIZABLE])
df_test['result_14'] = np.ones(len(traindf['close']))
#df_test['result_1'] = traindf['result_1']*2-1 # convert signal from true/false to 1/0

#use result_14 as the signal column, close price as execution price, initial fund $10,000, include commmision & fee
balance, profit, ROI = evaluate_profit(df_test, start_date, end_date, 10000, 'result_14', 'close', False)
print "ROI over {0} days: {1:.2f}%".format((end_date-start_date).days, ROI*100)

ROI over 1825 days: 67.65%


In [9]:
'''
Example for evaluate_profit() -- Baseline test on scaled testdf with inverse transform
'''
start_date = datetime.date(2015, 1, 1)
end_date = datetime.date(2015, 12, 31)
df_test = testdf.copy()
df_test[STANDARDIZABLE] = scaler.inverse_transform(df_test[STANDARDIZABLE])
df_test['result_14'] = np.ones(len(testdf['close']))
#df_test['result_1'] = traindf['result_1']*2-1 # convert signal from true/false to 1/0

#use result_14 as the signal column, close price as execution price, initial fund $10,000, include commmision & fee
balance, profit, ROI = evaluate_profit(df_test, start_date, end_date, 10000, 'result_14', 'close', False)
print "ROI over {0} days: {1:.2f}%".format((end_date-start_date).days, ROI*100)

ROI over 364 days: -3.47%


### Making baseline predictions

We'll create a dictionary `predictions` with keys the names of the models we use, and value the predictions on the test set. We'll also create a dictionary `predictions_valid` which saves the results of models on the validation set. This might seem a strange thing to do, but as we shall see later, this will come useful in ensembling

In [10]:
predictions={}
#predictions_valid={}
predictions_ROI={}

#### 1.3 Make the prediction from the `baseline` model

Store the predictions on the test set `testdf` in `predictions["baseline"]`, and on the validation set in `pedictions_valid` with the same key.

In [ ]:
#your code here
predicted=np.ones(len(testdf))

#for index, row in testdf.iterrows():
#    predicted[index]=ybar + user_biases[row["user_id"]] + item_biases[row["business_id"]]

predictions["baseline"] = predicted

#predicted=np.zeros(len(validatedf))

#for index, row in validatedf.iterrows():
#    predicted[index]=ybar + user_biases[row["user_id"]] + item_biases[row["business_id"]]

#predictions_valid["baseline"]=predicted

How did we do?

We define a function to calculate the root-mean-square error between actual values and predictions. We could have used the corresponding function in `sklearn` as well.

In [ ]:
def get_rmse(s, s_predict):
    diff = s - s_predict
    return np.sqrt(np.dot(diff,diff)/diff.shape[0])

We now write a `compare_results` function which plots the predicted star values against the actual ones. The x axis is the actual rating, so predicted results at any integer star value will be up and down the grid line at that value. We take the average of the ratings and plot a point there, and fill-inbetween one standard deviation on each side of this point. We also plot the 45 degree line for reference.

In [ ]:
def compare_results(stars_actual, stars_predicted, ylow=1, yhigh=6, model="", predicteds=False, onsame=False, axis=False):
    """
    plot predicted results against actual results. Takes 2 arguments: a
    numpy array of actual ratings and a numpy array of predicted ratings
    scatterplots the predictions, a unit slope line, line segments joining the mean,
    and a filled in area of the standard deviations."
    """
    if onsame:
        ax=onsame
    elif axis:
        ax=axis
    else:
        fig=plt.figure()
        ax=plt.gca()
    df=pd.DataFrame(dict(actual=stars_actual, predicted=stars_predicted))
    xp=[]
    yp=[]
    for k,v in df.groupby('actual'):
        xp.append(k)
        yp.append(v.predicted.mean())        
    
    cl, = ax.plot(xp,yp, 's-', label="means for %s" % model)
    c=cl.get_color()

    sig=df.groupby('actual').predicted.std().values
    ax.fill_between(xp, yp - sig, yp + sig, 
                 color=c, alpha=0.2)
    if predicteds:
        ax.plot(df.actual, df.predicted, '.', color=c, alpha=0.1, label="predicted for %s" % model)

    if not onsame:
        ax.plot([1,5],[1,5], 'k', label="slope 1")
        ax.set_xlabel("actual")
        ax.set_ylabel("predicted")
        ax.set_ylim([ylow,yhigh])
        ax.set_xlim([0.9, 5.1])
    ax.legend(frameon=False, loc="upper left")
    rmse=get_rmse(stars_actual, stars_predicted)
    print "RMSE for %s" % model, rmse
    return ax,rmse

We see how our prediction works using the baseline model.

In [ ]:
compare_results(testdf.stars,predictions['baseline'], model="baseline", predicteds=True);

How are we to interpret such diagrams?

Firstly, note that increasing bias in a model decreases the variation between the rating groups, resulting in a flatter blue curve. Here I am using the term bias in the sense of the bias-variance tradeoff. For example, the most biased model would be to leave out even the user and item "biases" (sorry for multiple uses of the term!) and just fit a flat average line, completely horizontal. Our prediction here is less biased than that.

The second thing to notice is that the precision as measured by the width of the grey region measures the variance WITHIN rating groups, not between them. 

## Q2 Collaborative Filtering using kNN

### Setting up the Collaborative Filtering Model

Can we do a better job of predicting the rating $Y_{um}$ user $u$ would give to restaurant $m$? According to the central dogma of CF, we ought to be able to use the responses of *similar* users regarding *similar* restaurants to get a better prediction. 

We can make an estimate of $Y_{um}$ as:

$$ \hat{Y_{um}} = \hat Y_{um}^{baseline}\, + \,\frac{\sum\limits_{j \in S^{k}(m)} s_{mj} ( Y_{uj} - \hat Y_{uj}^{baseline} )}{\sum\limits_{j \in S^{k}(m)} s_{mj} } $$

where $S^{k}(m)$ is the $k$ neighbor items of item $m$ based on some *pooling criterion*, for example, those items which have been rated by user $u$.


To do this, we compute a *similarity measure* $s_{mj}$ between the $m$th and $j$th items. This similarity might be measured via [cosine similarity](http://en.wikipedia.org/wiki/Cosine_similarity), [pearson co-efficient](http://en.wikipedia.org/wiki/Pearson_product-moment_correlation_coefficient) or using other distance based measures. Here we shall use the Pearson coefficient. This measures the tendency of users to rate items similarly. Since most ratings are unknown, it is computed on the "common user support" (`n_common`), which is the set of common raters of both items. This is why we spent some time earlier trying to craft a data set which was doable in reasonable time in the context of this homework but still having reasonable values of `n_common`.

#### Global Recommender

**$S^{k}(m)$ depends on items only.**

Now we have a way to pool information between similar restaurants to try to predict a user's recommendation. But how do we choose the neighborhood to pool over? The simplest choice is to calculate the similarity between items using their entire common user support, and rank the nearest neighbors of an item by this similarity. We call this a "global" recommender because it assumes that every user perceives the similarity between restaurants in the same way. 
The global recommender does have the advantage of dealing with the possible sparsity of the user's rated items, but also the disadvantage of giving one answer for all users, without taking the user's preferences into account. This is a classic case of bias-variance tradeoff.

The above recommender also has a second problem: we have no way of telling how good our results are. The problem is this:

When I compute the global neighborhood of a restaurant, one of the restaurants in these `k` nearest neighbors might not have a rating from me in the training set, in the usual training-validation paradigm. So I cant compute that $Y_{uj}$ on the right hand side in the formula above. The best I could do is to use the average rating of restaurant in the training set. I can, of-course, still calculate the nearest neighbors and make recommendations based on the distances I calculate. 

We shall not do this recommender here. See HW4 in the 2013 cycle of cs109 for an example of such a recommender. But, you might see such a recommender in the wild: its then a good faith recommender, as it is very hard to validate such a recommender without playing games with averages.

#### Local recommender

**$S^{k}(m) = S^{k}(m;u)$ depends on items and the user in question **

The aim is to get more fine-grained predictions about each user, and try to predict what _rating_ a user would give to a restaurant they have never tried before. To do this, we will try to personalize the information we use even further, and only pool information from restaurants that the user has rated.

This allows us to return to the original problem of prediction $Y_{um}$ for a restaurant $m$ that user $u$ has never rated before. Using our newly computed similarity metrics, we can modify our original baseline estimate by pulling in information from the user's neighborhood of the restaurant $m$, and predict $Y_{um}$ as:

$$ \hat{Y_{um}} = \hat Y^{baseline}_{um}\, + \,\frac{\sum\limits_{j \in S^{k}(m;u)} s_{mj} ( Y_{uj} - \hat Y^{baseline}_{uj} )}{\sum\limits_{j \in S^{k}(m;u)} s_{mj} } $$

where $s^{k}(m;u)$ is the $k$ neighbor items of item $m$ which have been rated by user $u$.

Now, this is not a particularly good assumption, especially in the situation where a restaurant is new (new item problem) or a user is new (cold start problem), or in the case when there are very few reviewers of a restaurant, or very few reviews by a user respectively. However, one must start somewhere!

Notice that in adding in the similarity term, we subtract the baseline estimate from the observed rating of the user's neighbor items.

### Common Support

Lets now make a histogram of the common user support (the number of common reviewers) of each pair of restaurants on the training set, and print the mean. We saw a function do this earlier when we created the smaller data set in `smalldf`. 

The common support is an important concept, as for each pair of restaurants, it is the number of people who reviewed both. It will be used to modify similarity between restaurants. If the common support is low, the similarity is less believable, and we would want to "regularize" it by some average estimate.

In this incarnation we create a two dimensional array `supports` which uses the `uiidmap`, and a Pandas Series `users_for_restaurants` with index the restaurant ids which will come useful later. We obtain the common reviewers as an intersection between two restaurants reviewers. We take care to fill the entire matrix including the diagonal. (This is the second implementation of a support finding routine you have seen: we use set intersections here; earlier we used boolean ANDs on a mask). You will write a third one, soon :-)

In [ ]:
#for each restaurant get the unique userids who rated it
users_for_restaurants = traindf.groupby('business_id').user_id.unique()
restaurants=uiids
lres=len(restaurants)
supports=[[[] for i in range(lres)] for i in range(lres)]
supporthistlist=[]
for i,rest1 in enumerate(restaurants):
    for j,rest2 in enumerate(restaurants):
        if  i <= j:#its symmetric
            if rest1==rest2:
                common_reviewers=users_for_restaurants[rest1]
            else:
                common_reviewers = set(users_for_restaurants[rest1]).intersection(set(users_for_restaurants[rest2]))
                supporthistlist.append(len(common_reviewers))
            supports[i][j]=common_reviewers
            supports[j][i]=common_reviewers
print "Mean and Median support is:",np.mean(supporthistlist), np.median(supporthistlist)

We see that the common support is a bit lower on the training set due to the loss of some ratings to the validation and test sets.

### Calculating similarity

Remember that even though each reviewer uses the same 5-star scale when rating restaurants, comparing two users by comparing their raw user ratings can be problematic. Consider a user whose average rating is 2. This is a curmudgeonly user. Consider another whose average rating is 4. This is a rather enthusiastic one. How should we compare a 3 rating by the curmudgeonly one to a 5 rating of the enthusiastic one? This is why we computed the baseline model in the last problem.

#### 2.1 Write a function `pearson_sim` that calculates the pearson similarity

We must subtract the average rating of the user from the actual rating of the restaurants in computing the similarity of two restaurants. This makes the ratings by the two users comparable. We do this in the function `pearson_sim` defined below.

If there is no common support (`n_common=0`), we have no basis for making a similarity estimate, and so we set the similarity to 0. In the case that the individual restaurant rating variance is 0, such as in the case where there is only one common reviewer (`n_common=1`), convert the `NaN` that the scipy `pearsonr` returns to`0.`.

In [ ]:
from scipy.stats.stats import pearsonr
"""
Given a subframe or dictionary of restaurant 1 reviews and a subframe or dictionary of restaurant 2 reviews, 
<strong>and the number of common reviewers n_common</strong>,
where the reviewers are those who have reviewed both restaurants, return 
the pearson correlation coefficient between the user average subtracted ratings.
The case for zero common reviewers is handled separately<strong>, return 0</strong>. If the correlation is
NaN if any of the individual variances are 0 (the n=1 case), return 0 instead.
"""
#your code here
def pearson_sim(rest1_reviews, rest2_reviews, n_common):
    if n_common==0:
        return 0
    testp= (pearsonr(rest1_reviews['stars']-rest1_reviews['user_avg'], 
                     rest2_reviews['stars']-rest2_reviews['user_avg'])[0])
    if np.isnan(testp):
        return 0
    return testp

#### Get Restaurant reviews

The function `get_restaurant_reviews` defined below takes a restaurant `business_id` and a set of users, and returns the reviews of that restaurant by those users. You will use this function in calculating a similarity function below. The set of users is all the users that rated a restaurant for the global recommender, and later, a restricted set for the local recommender.

In [ ]:
def get_restaurant_reviews(restaurant_id, df, set_of_users):
    """
    given a resturant id and a set of reviewers, return the sub-dataframe of their
    reviews.
    """
    mask = (df.user_id.isin(set_of_users)) & (df.business_id==restaurant_id)
    reviews = df[mask]
    reviews = reviews[reviews.user_id.duplicated()==False]
    return reviews

### Making a database of similarities

We now move to calculating a global database of pairwise restaurant similarities.
We provide you here with a Class to make a database of the similarities for each pair of restaurants in the database. 

The class `Database` is initialized in its constructor `__init__` . A `class` is a construct which lets you store state in a program. Here we create a array for the database and fill it with zeros. 

The `set_supports` method is used to store the `supports` that we computed earlier.

The `get` method on the database can be used to retrieve the similarity for two business ids.

The general strategy for filling in this database is as follows:

1. For each of the two restaurants, we get the set of reviewers who have reviewed the restaurant and compute the intersection of these two sets. We also compute the number of common reviewers `n_common`.

2. We get the reviews for each restaurant as made by these common reviewers and get the star ratings and user averages. 

3. We calculate the similarity using `similarity_func`.

4. We return the similarity and `n_common` in a tuple `(sim, n_common)`. If the similarity is a `NaN`, set the similarity to 0.

In [ ]:
class Database:
    "A class representing a database of similarities and common supports"
    
    def __init__(self, rindexmap, supports):
        "the constructor, takes a map of restaurant id's to integers"
        database={}
        self.rindexmap=rindexmap
        self.supports=supports
        l_keys=len(self.rindexmap.keys())
        self.database_sim=np.zeros([l_keys,l_keys])
        self.database_sup=np.zeros([l_keys, l_keys], dtype=np.int)

    def set_supports(self, supports):
        self.supports=supports
        
    def get(self, b1, b2):
        "returns a tuple of similarity,common_support given two business ids"
        sim=self.database_sim[self.rindexmap[b1]][self.rindexmap[b2]]
        nsup=self.database_sup[self.rindexmap[b1]][self.rindexmap[b2]]
        return (sim, nsup)

Lets create an instance of the `Database class`, using the `supports` and `uiidmap` we calculated when making the histogram of common supports and even earlier

In [ ]:
db=Database(uiidmap, supports)

### Map reduce to populate database

We shall fill in the database using a map-reduce computation, which can, if implemented in mrjob or Spark, be scaled over multiple machines. Here we just wish to illustrate this process, and right our own implementation.

We display here, an outline of the implementation, so that the steps in the process can be followed:

```python
def map_reduce(tuples)
    mapped1=map(mapper1, tuples)
    combine1=combiner(mapped1)
    reduced1=reduce(lambda x,y: x + [reducer1(y)], combine1, [])
    mapped2=map(mapper2,reduced1)
    combine2=combiner_list(mapped22)
    output=reduce(lambda x,y: x + [reducer2(y)], combine2, [])
    return output
tuples=traindf.itertuples()
sims=map_reduce(tuples)
```

In [ ]:
print traindf.columns
for row in traindf.head(2).itertuples():
    print row

Note the additional index at the beginning of the tuple.

**(1)** In `mapper1`, we first take *each* tuple in the form above from the dataframe, and apply the function using python's `map` to spit out a tuple of the form: `user_id, (business_id, stars, user_avg)`. `mapped1` is then a list of these tuples.

`mapped1=map(mapper1, tuples)`

In [ ]:
def mapper1(row):
    return row[1], (row[2], row[5], row[14])

**(2)** Then, in `combiner` we do what a map reduce framework like hadoop does for us and "collect" all the tuples belonging to a given `user_id` to form a list `combine1` of the form `[(user_id, list of (business_id, stars, user_avg) ),...]`

`combine1=combiner(mapped1)`

In [ ]:
def combiner(items):
    indict={}
    for key, value in items:
        if not indict.has_key(key):
            indict[key]=[]
        indict[key].append(value)
    return indict.items()

**(3)** Then, in `reducer1`, we use python's `reduce` function to reduce the list we made above into another list `reduced1`. This list is made by starting from the empty list `x=[]`. Then for each element `y` in `combine1`, we return a tuple of `user_id` with a list of tuples `[(business_id,(stars, user_avg)),...]`. Thus `reduced1` is a list of tuples `[(user_id, [(business_id,(stars, user_avg)),...]),...]`.

`reduced1=reduce(lambda x,y: x + [reducer1(y)], combine1, [])`

In [ ]:
def reducer1(the_input):
    user_id, values = the_input
    ratings=[]
    for business_id,stars,user_avg in values:
        ratings.append((business_id,(stars, user_avg)))
    return user_id, ratings

**(4)** Its now your job to write the next mapper `mapper2`:

#### 2.2 Write `mapper2`

`mapped2=map(mapper2,reduced1)`

`mapper2` takes each element of the list `reduced1`, that is, something that looks like `(user_id, [(business_id,(stars, user_avg)),...])`. Use the function `combinations_with_replacement` from python's itertools module, and comes up with all possible 2-combinations of elements from the list `[(business_id,(stars, user_avg)),...]` (including those which repeat the element. You will notice that what this does is to output every combination of restaurant_id's that the user in question has reviewed.

So, ignoring the user_id key, take all combinations of business pairs and return a list whose items are tuples. Each tuple has 2 elements. The first element is a tuple of pair ids, and the second tuple is a tuple of pair rating information. The first tuple looks something like `(biz1_id, biz2_id)`, while the second one looks like `((stars1, user_avg1),(stars2, user_avg2))`. Make sure that if `biz1id > biz2id` lexicographically (which is a big way of saying alphabetically: using the alphabets imposes an ordering on restaurant ids), you reverse the order of both the tuples. What this does is that it ensures that the first business id is always smaller than or equal to the second one and thus we dont do extra work.

In [ ]:
#your code here
#per the reference in the instructions, much logic gleaned from: HW4 in the 2013 cycle of cs109
from itertools import combinations_with_replacement
def mapper2(the_input):
    user_id, ratings = the_input
    for biz1, biz2 in combinations_with_replacement(ratings, 2):
        biz1id, biz1rat=biz1
        biz2id, biz2rat=biz2
        if biz1id <= biz2id :
            yield (biz1id, biz2id), (biz1rat, biz2rat)
        else:
            yield (biz2id, biz1id), (biz2rat, biz1rat)

**(5)** We now run `mapper2` on the entire list `reduced1` and as before, run a combiner to *collect* all tuple-pairs with the first tuple as the key. This will now give us, for each business id pair, a list of `((stars1, user_avg1),(stars2, user_avg2))` tuples, and thus, a list of the common support of each business id pair, in the form `[(biztuple, list of ((stars1, user_avg1),(stars2, user_avg2))),...]` which we now store in `combine2`.

`combine2=combiner_list(mapped22)`


In [ ]:
def combiner_list(itemslist):
    indict={}
    for items in itemslist:
        for key, value in items:
            if not indict.has_key(key):
                indict[key]=[]
            indict[key].append(value)
    return indict.items()

**(6)** Now we write the next reducer `reducer2`.

#### 2.3 Write `reducer2`

From `combiner_list`, for each business_id pair we get `common_ratings` which is a list of tuples with ratings by the same user for the two businesses in the form `[((stars1, user_avg1),(stars2, user_avg2)),..]`. Write a function `reducer2` that takes the output of `combiner_list` and uses these common ratings to return, for each restaurant pair, two tuples. The first tuple if a tuple of restaurant ids, while the second tuple is their pearson correlation and their common support. In other words, the last line of this reducer function ought to look something like `return (restaurant1_id, restaurant2_id), (rho, n_common)`.
Given that the output 

HINT: By converting these to dictionaries with values that are `np.array`s we can use the same `pearson_sim` similarity function we defined above.

In [ ]:
#your code here
#per the reference in the instructions, much logic gleaned from: HW4 in the 2013 cycle of cs109
def reducer2(items):
    (rest1, rest2), common_ratings = items
    stars1=[]
    uavgs1=[]
    stars2=[]
    uavgs2=[]
    n_common=0

    for rt1, rt2 in common_ratings:
        stars1.append(float(rt1[0]))
        uavgs1.append(float(rt1[1]))
        stars2.append(float(rt2[0]))
        uavgs2.append(float(rt2[1]))
        n_common+=1

    rho=pearson_sim({"stars":np.asarray(stars1),"user_avg":np.asarray(uavgs1)},
                    {"stars":np.asarray(stars2),"user_avg":np.asarray(uavgs2)},
                    n_common)

    return (rest1, rest2), (rho, n_common)

Ok, so we create the `map_reduce` function for real and then run it!

In [ ]:
def map_reduce(tuples):
    mapped1=map(mapper1, tuples)
    combine1=combiner(mapped1)
    reduced1=reduce(lambda x,y: x + [reducer1(y)], combine1, [])
    mapped2=map(mapper2,reduced1)
    combine2=combiner_list(mapped2)
    output=reduce(lambda x,y: x + [reducer2(y)], combine2, [])
    return output

In [ ]:
tuples=traindf.itertuples()
sims=map_reduce(tuples)

We now populate our instance of the database class by passing the instance `db` to the function `populate_from_mr` below. It takes the output of the reducer, finds the integer indices corresponding to the business id's, and populates the slots in the database.

In [ ]:
def populate_from_mr(db, df, mapredlist):
    for tpair,vpair in mapredlist:
        i1=db.rindexmap[tpair[0]]
        i2=db.rindexmap[tpair[1]]
        db.database_sim[i1][i2]=vpair[0]
        db.database_sup[i1][i2]=vpair[1]
        db.database_sim[i2][i1]=vpair[0]
        db.database_sup[i2][i1]=vpair[1]

In [ ]:
populate_from_mr(db, traindf, sims)

#### Checking our work

To check your work, since this is your first exposure to map-reduce, we provide you functions to do this the usual way using numpy and pandas. We first create a second instance of the database.

In [ ]:
db2=Database( uiidmap, supports)

The function `populate_by calculating` iterates over every possible pair of business_id's in the dataframe and populates the database with similarities and common supports. It takes as arguments a function the similarity function `similarity_func` like `pearson_sim` (`calculate_similarity` uses this to calculate the similarity).

We write the function `calculate_similarity` inside this class that is the workhorse method that we use to populate the database. This method operates between two restaurants and calculates a similarity for them, taking a dataframe and a similarity function `similarity_func`. 

In [ ]:
def calculate_similarity(db, df, rest1, rest2, similarity_func):
    # find common reviewers
    common_reviewers = db.supports[db.rindexmap[rest1]][db.rindexmap[rest2]]
    n_common=len(common_reviewers)
    if rest1==rest2:
        return 1., n_common
    #get reviews
    rest1_reviews = get_restaurant_reviews(rest1, df, common_reviewers)
    rest2_reviews = get_restaurant_reviews(rest2, df, common_reviewers)
    sim=similarity_func(rest1_reviews, rest2_reviews, n_common)
    return sim, n_common

def populate_by_calculating(db, df, similarity_func):
    """
    a populator for every pair of businesses in df. takes similarity_func like
    pearson_sim as argument
    """
    items=db.rindexmap.items()
    for b1, i1 in items:
        for b2, i2 in items:
            if i1 <= i2:
                sim, nsup=calculate_similarity(db, df, b1, b2, similarity_func)
                db.database_sim[i1][i2]=sim
                db.database_sim[i2][i1]=sim
                db.database_sup[i1][i2]=nsup
                db.database_sup[i2][i1]=nsup

We run the `populate_by_calculating` function.

In [ ]:
%%time
populate_by_calculating(db2, traindf, pearson_sim)

Lets check it for a pair of results. Try some other tests. *These specific two may not be there in the training set so you will need to find your own test businesses.*

In [ ]:
tpair=('FV0BkoGOd3Yu_eJnXY15ZA', 'O-Xa9GCFWI65YiBD5Jw_hA')
print db.get(tpair[0],tpair[1]),db2.get(tpair[0],tpair[1])

In [ ]:
print db.get(tpair[1],tpair[0]),db2.get(tpair[1],tpair[0])
print db.get(tpair[0],tpair[0]),db2.get(tpair[0],tpair[0])

#### Shrunk similarities

We are now going to find the k-nearest restaurants to a given restaurant based on the database of similarities that we calculated. But we have a problem.

Consider the two cases where there are two common reviewers, and where there are 40. In the former case, we might get a artificially high similarity based on the tastes of just this user, and thus we must reduce its importance in the nearest-neighbor calculation. In the latter case, we would get a much more unbiased estimator of the similarity of the two restaurants.

To control the effect of small common supports, we can **shrink** our pearson co-efficients. We shall do this by using the "regularization" parameter `reg`:

$$s_{mj} = \frac{N_{common}\, \rho_{mj}}{N_{common}+reg} $$

where $N_{common}$ (`n_common`) is the common reviewer support and $\rho_{ij}$ is the pearson co-relation coefficient.

Recall the notions of regularization introduced in class. We want to reduce the variance in our estimates, so we pull our estimates in toward a conservative point in a way that strongly corrals in estimates when there is very little data, but allows the data to speak when there is a lot. 

We define a function `shrunk_sim` which takes a `sim` and `n_common`, and shrinks the similarity down using the regularizer `reg`.

In [ ]:
def shrunk_sim(sim, n_common, reg=3.):
    "takes a similarity and shrinks it down by using the regularizer"
    ssim=(n_common*sim)/(n_common+reg)
    return ssim

### Calculating Nearest Neighbors

Now we can move to writing a `knearest` function, which finds the `k` nearest neighbors of a given restaurant based on the shrunk similarities we calculate. Note that as defined here, the nearest neighbors are restricted to the restaurants a user has reviewed.

#### Similarity Distances

The very idea of nearest neighbor should engender in you the idea of distances; and distances should not be negative.
Furthermore, you would not be able to use distances other than those implied by the correlation coefficient in the calculation of nearest neighbors (such as a Manhattan distance or Jacard similarity).

We can fix this for the case of pearson coefficient by just a simple rescaling to the 0-1 range!

$$ \rho \rightarrow \frac{1 - \rho}{2} $$.

Notice this quantity is always positive and between 0 and 1. Its 0 at full correlation, and 1 at full anticorrelation.

#### We write a function `knearest` 

This function returns a *k-length sorted list* of 3-tuples each corresponding to a restaurant, the k-nearest restaurants to one we ask for.

The tuple structure is `(business_id, shrunken distance, common support)` where the distance and common support are with respect to the restaurant whose neighbors we are finding, and the `business_id` is the id of the "nearby" restaurant found. The nearby restaurants are found from a supplied numpy array of restaurants `set_of_restaurants`. We use `itemgetter` from the `operator` module to do the sorting. 

In [ ]:
"""
Function
--------
knearest

Parameters
----------
restaurant_id : string
    The id of the restaurant whose nearest neighbors we want
set_of_restaurants : array
    The set of restaurants from which we want to find the nearest neighbors
dbase : instance of Database class.
    A database of similarities, on which the get method can be used to get the similarity
  of two businesses. e.g. dbase.get(rid1,rid2)
k : int
    the number of nearest neighbors desired, default 7
reg: float
    the regularization.
    
  
Returns
--------
A sorted list
    of the top k similar restaurants. The list is a list of tuples
    (business_id, shrunken similarity, common support).
"""
from operator import itemgetter
def knearest(restaurant_id, set_of_restaurants, dbase, k=7, reg=3.):
    """
    Given a restaurant_id, dataframe, and database, get a sorted list of the
    k most similar restaurants from the set of restaurants.
    """
    similars=[]
    for other_rest_id in set_of_restaurants:
        if other_rest_id!=restaurant_id:
            sim, nc=dbase.get(restaurant_id, other_rest_id)
            ssim=shrunk_sim(sim, nc, reg=reg)
            simdist=(1. - ssim)/2.
            similars.append((other_rest_id, simdist, nc ))
    similars=sorted(similars, key=itemgetter(1))
    return similars[0:k]

We write a function `get_users_restaurants` to get the restaurants a user has rated, and what rating he or she gave them, in preparation for the prediction step.

In [ ]:
def get_users_restaurants(df, user_id):
    dfuser=df[df.user_id==user_id]
    dfuserdedup=dfuser.drop_duplicates('business_id')
    return dict(zip(dfuserdedup.business_id.values, dfuserdedup.stars.values))

#### 2.4 Write a function to calculate the predicted rating for a user and restaurant

Now write a function that returns the predicted rating for a user and an item using the formula at the beginning of this problem. Include code to deal with the possibility that the sum of scores that goes in the denominator is 0: return an predicted rating of the baseline portion of the formula in that case. This function `rating` takes as arguments the `train_map` (for e.g., see `train_avg` from earlier which stores the baseline estimates of intercept (overall mean), user bias, and item bias), the database, the `set_of_restaurants`, the wanted `restaurant_id` and `user_id`, and `k` as well as the regularizer `reg`.

In [ ]:
"""
Function
--------
rating

Parameters
----------
set_of_restaurants: Dictionary
    The dictionary of restaurant: star-rating pairs you want to make the prediction from.
    This would be the output of a function like get_users_restaurants
train_map: Dictionary
    A dictionary with keys mean, users and items which have estimates of
    overall average or intercept, user coefficients(averages), and
    item coefficients(averages) respectively
dbase : instance of Database class.
    A database of similarities, on which the get method can be used to get the similarity
  of two businessed. e.g. dbase.get(rid1,rid2)
restaurant_id : string
    The id of the restaurant whose nearest neighbors we want
user_id : string
    The id of the user, in whose reviewed restaurants we want to find the neighbors
k : int
    the number of nearest neighbors desired, default 7
reg: float
    the regularization.
    
  
Returns
--------
A float
    which is the imputed rating that we predict that user_id will make for restaurant_id
    
Notes
--------
If the sum of scores is 0, return the baseline estimate of the ranking.
"""
#your code here
#per the reference in the instructions, much logic gleaned from: HW4 in the 2013 cycle of cs109
def rating(set_of_restaurants, train_map, dbase, restaurant_id, user_id, k=7, reg=3.):
    mu=train_map["mean"]
    #users_reviews=df[df.user_id==user_id]
    nsum=0.
    scoresum=0.
    nears=knearest(restaurant_id, set_of_restaurants, dbase, k=k, reg=reg)
    restaurant_mean=train_map["items"][restaurant_id]
    user_mean=train_map["users"][user_id]
    scores=[]
    #print nears
    for r,s,nc in nears:
        scoresum=scoresum+s
        scores.append(s)
        #r_reviews_row=users_reviews[users_reviews['business_id']==r]
        r_stars=set_of_restaurants[r]#train_map["items"][r]#r_reviews_row.stars.values[0]
        r_avg=train_map["items"][r]#train_map["users"][user_id]#r_reviews_row.business_avg.values[0]
        rminusb=(r_stars - (r_avg + user_mean - mu))
        nsum=nsum+s*rminusb
    baseline=(user_mean + restaurant_mean - mu)
    #we might have nears, but there might be no commons, giving us a pearson of 0
    if scoresum > 0.:
        val =  nsum/scoresum + baseline
    else:
        val=baseline
    return val

### Predicting on a user's neighborhood

#### An example

Here is an example of how the prediction works:

In [ ]:
trainuser=traindf.loc[0].user_id
testrest=testdf[testdf.user_id==trainuser].business_id.values[0]
print trainuser, testrest

In [ ]:
def get_actual(df, userid, bizid):
    return df[(df.user_id==userid) & (df.business_id==bizid)]['stars'].values[0]

print "Actual", get_actual(testdf, trainuser, testrest)
print "Predicted",rating(get_users_restaurants(traindf, trainuser), train_avgs, db, testrest, trainuser, k=2, reg=3.)

We now write a function which uses the rating function you defined to make a set of predictions on the validation set. Note the default values for the `k` nearest neighbors and regularization `reg`. This function takes an input dataframe `indf`, typically the validation or test dataframe, and iterated over the ratings in this frame, using the `rating` function you wrote to make predictions. It returns a tuple `preds, actuals`. Each of the variables represents an array of ratings from `indf`, with `preds` being the predictions we made and `actuals` being the actual ratings.

In [ ]:
def get_ratings_user_nbd(indf, traindf, train_map, db, k=2, reg=3.):
    zips=zip(indf.business_id, indf.user_id, indf.stars)
    preds=[]
    actuals=[]
    for (r,u,actual) in zips:
        pred=rating(get_users_restaurants(traindf, u),train_map, db, r,u, k, reg)
        preds.append(pred)
        actuals.append(actual)
    return np.array(preds), np.array(actuals)

To show the usage of this function, lets set $k$ and `reg` to 4 and use it on the training set itself so that we can see what overfitting looks like. Since the training data frame is large, this might take a minute to run.

In [ ]:
pt, at = get_ratings_user_nbd(traindf, traindf, train_avgs, db, k=4, reg=4.)
compare_results(at,pt, model="knn(user) on training k=4, reg=4", predicteds=True)

Note the really low RMSE.

#### 2.5 Plot graphs for a 2x2 grid of `k` and `reg` and interpret your results

Run four different models, for the combinations of parameters from `k=[2,15]` and `reg=[1.,15.]`. For each model, use `compare_results` to plot a graph of the results. Use the `axis` keyword argument of `compare_results` to plot these on a 2x2 grid. Interpret your results from both the rmse of the models and the graphs.

In [ ]:
#your code here
fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(12,12), tight_layout=True)

pt, at = get_ratings_user_nbd(traindf, traindf, train_avgs, db, k=2, reg=1.)
compare_results(at,pt, model="knn(user) on training k=2, reg=1", predicteds=True, axis=axes[0][0])
pt, at = get_ratings_user_nbd(traindf, traindf, train_avgs, db, k=2, reg=15.)
compare_results(at,pt, model="knn(user) on training k=2, reg=15", predicteds=True, axis=axes[0][1])
pt, at = get_ratings_user_nbd(traindf, traindf, train_avgs, db, k=15, reg=1.)
compare_results(at,pt, model="knn(user) on training k=15, reg=1", predicteds=True, axis=axes[1][0])
pt, at = get_ratings_user_nbd(traindf, traindf, train_avgs, db, k=15, reg=15.)
compare_results(at,pt, model="knn(user) on training k=15, reg=15", predicteds=True, axis=axes[1][1])

fig.show()

###### Interpret your results
At a low k, the predictions seem to follow the "actual" line more closely. At higher k's, the width of the variance (light blue) seems to get narrower. Also, the higher regularization tends to yield lower RMSE figures.

#### 2.6 Validate over `k` and `reg`

Now use this function defined in a Validation loop over different values of `k` between 1 and 100 and `reg` between 1 and 100. (Note: this is validation, not cross-validation). The validation loop will take a few minutes to run. Start with a coarse loop, but then fine grain it down to be within 5 on both parameters. Store in a dictionary `rmsedict`, keyed by the tuple `(k, reg)` the values of the root mean square error between the predictions and the actual values.

Depending on how many points you have on your gris, this might take a few minutes to run.

In [ ]:
#your code here
rmsedict={}
for k in np.arange(1,100,5):
    for reg in np.arange(1.,100.,5.):
        pt, at = get_ratings_user_nbd(traindf, traindf, train_avgs, db, k, reg)
        rmsedict[(k,reg)]=get_rmse(at, pt)

Note the general flatness of the minimization surface in the parameter space.

We now get the values of the `k` and `reg` which minimize the root mean square error.

In [ ]:
mintup=min(rmsedict, key=rmsedict.get)
mintup

In [ ]:
for k in np.arange(2,11,1):
    for reg in np.arange(97.,101.,1.):
        pt, at = get_ratings_user_nbd(traindf, traindf, train_avgs, db, k, reg)
        rmsedict[(k,reg)]=get_rmse(at, pt)

In [ ]:
mintup=min(rmsedict, key=rmsedict.get)
mintup

#### 2.7 Use `mintup` to calculate and plot the results on the test set and compare the results with the baseline model

Make predictions on the test and validation set. Store these in `predictions['knn']` and `predictions_valid['knn']` respectively. Plot the results on the test set and explain them. Compare the results against the  baseline model. Explain your results and how you think it will change with test and validation sets having reviews of restaurants with more support and users with more rankings.

In [ ]:
#your code here
tpt, tat = get_ratings_user_nbd(testdf, traindf, train_avgs, db, k=3, reg=98.)
print get_rmse(tat, tpt)
predictions["knn"]=tpt
vpt, vat = get_ratings_user_nbd(validatedf, traindf, train_avgs, db, k=3, reg=98.)
print get_rmse(vat, vpt)
predictions_valid["knn"]=vpt

In [ ]:
ax,rmse=compare_results(tat,tpt, model="knn(user) on test k=3, reg=98", predicteds=True)
compare_results(testdf.stars,predictions['baseline'], model="baseline", onsame=ax, predicteds=True);

###### Comments
Intuitively, I would have expected knn results to be better than the baseline. But that's not what the RMSE or graph is showing. Restaurants with more support and users with more rankings should allow for better predictions.

### How to interpret these results?

In other words, how would you tell a user where to go eat?


In [ ]:
randomuser=np.random.choice(uuids)
randomrest=np.random.choice(uiids)

set_of_restaurants = get_users_restaurants(traindf, randomuser)
tops=knearest(randomrest, set_of_restaurants, db, k=mintup[0], reg=mintup[1])
print "For restaurant",randomrest, " and user", randomuser, ", top matches are:"
for i, (biz_id, sim, nc) in enumerate(tops):
    print i,biz_id, "| Sim", sim, "| Support",nc,"| Actual", get_actual(traindf, randomuser, biz_id)

One can also find the top recommendations for a user, for example. See 2013 homework for this.

## Q3: The Baseline model with regularization

The model we used in the last section was:

$$ \hat Y_{um}^{baseline} = \hat \mu + \hat \theta_{u} + \hat \gamma_{m} $$

where we estimated $\hat \theta_{u}$ as $\bar Y_u - \bar Y$ and $\hat \gamma_{m}$ as $\bar Y_m - \bar Y$.

### Writing it as a Ridge Regression

In this section we'll get more sophisticated and estimate these quantities as regularized regressions.

Look at the formula above. Notice that there is a constant term, $\hat \mu$, which looks like an intercept, and $N_{user}=U$ plus $N_{item}=M$ terms that dont exactly look like coefficients of a regression.

There are two key observations to be made which help us cast this problem as a regression. The first one is that we can flatten the non-zero elements of the recommendation matrix $ \hat Y_{um}^{baseline}$ into a column vector: 

![representation](representation.png)
The second key observation to make to recast the above equation into the following form:

$$ Y_{um}^{baseline} =  \mu + \bar \theta \cdot I_{u} +  \bar \gamma \cdot I_{m} $$

where $I_{u}$ and $I_{m}$ are indicator variables for the uth user and mth item that go into the feature matrix. Here $\bar \theta$ means the vector of all thetas, ie a vector of all the coefficients for users who have made ratings in our training set. Similarly $\bar \gamma$ is a vector of coefficients for all the items for which ratings have been made in our training set. In our previous model these coefficients were simply the sample means we used.

The indicator $I_{u}$ is a feature vector that takes on value 1 if it is user $u$ and 0 otherwise. We stack this horizontally with the indicator for item $m$ rated to create a row of the feature matrix. In other words, the row of our feature matrix for the regression for  $Y_{um}^{baseline} $ looks like

`1 (intercept),0(user 1),0(user 2),..,1(user u),..0(user N_users),0(item 1),0(item 2),..,1(item m),..,0(item N_items)`

This can be seen in the diagram below.
The left hand red column vector is the response, or the flattened-to-vector recommendation matrix. Note that now we are flattening only the known responses, and among then, even, only those on the training set. The right hand side has the intercept term (which used to be the baseline mean) plus the feature matrix multiplied by the co-efficients vector.  

And each row of this matrix consists of the horizontally stacked Indicator row vectors.  For example, if we are talking about the user with index 3 and item with index (ie, for $Y^{baseline}_{35}$), the first indicator vector has a 1 at the third (index 2) position and the second indicator has a 1 at the 5th (index 4) position.Then we put these side by side to create the entire row, moving on to do the same thing for the entire next row.

![expanding](expanding.png)

Now we are in business and can do a simple regression. 


BUT.

Notice that we have $U + M + 1$ coefficients to fit. Thats a lot. And in a problem that is very sparse..most of the actual observations of rating of a restaurant by a user simply dont exist. Users rate only a few restaurants each. All of this should remind you of the dreaded word of Machine Learning: Overfitting.

But we know how to deal with overfitting. We use Regularization. 

Thus we modify our model to:

$$  Y_{um}^{baseline} =  \mu +  \bar \theta \cdot I_{u} +  \bar \gamma \cdot I_{m} $$

such that we minimize:

$$\sum_{u,m} \left( Y_{um} -  \mu - \bar \theta \cdot I_{u} -  \bar \gamma \cdot I_{m} \right)^2 + \alpha \left( \theta_{u}^2 + \gamma_{m}^2 \right)$$

Now, this is exactly the form of a ridge regression on the feature matrix with horizontally stacked indicator vectors!

So let us set up the feature matrix (also called design matrix) for a ridge regression. In `sklearn` we dont need to model the intercept in the design matrix so the size of the design matrix is the number of samples in the training set times the sum of unique users and unique items.

In [ ]:
from sklearn.linear_model import Ridge
features=np.concatenate([uuids,uiids])
features.shape

To construct the matrix, we take our dataframe, and set up a matrix of size $n_{samples} \times n_{features}$ and fill it with zero. We also set up the response $Y_{um}$. We then put a 1 in the design matrix for the $u$ and $m$ corresponding to the specific rating (thus two 1's in each sample row, all others 0's).

#### 3.1 Write a function to produce the design matrix

Write a function `getmats` which takes an input dataframe `indf` and outputs a tuple `designmatrix, ratings`. The `ratings` are the ratings from the `stars` column of the dataframe, in the order they were present in the dataframe. The design matrix is as described above, keeping this order in mind. HINT: using the `uiids` and `uuids` lists or the corresponding maps may be useful here.

In [ ]:
"""
Function
--------
getmats

Parameters
----------
indf: Dataframe
    The input dataframe
  
Returns
--------
tuple:
(
designmatrix
    which is the design matrix of samples X features
ratings
    which is the stars column of the input dataframe
)
"""
#your code here
def getmats(indf):
    ratings=indf["stars"].values
    designmatrix=np.zeros((indf.shape[0],features.shape[0]), dtype=np.int)
    for index, row in indf.iterrows():
        wrow = np.where(indf["review_id"]==row["review_id"])[0]
        ucol = np.where(features==row["user_id"])[0]
        bcol = np.where(features==row["business_id"])[0]
        designmatrix[wrow,ucol] = 1
        designmatrix[wrow,bcol] = 1
    return (designmatrix, ratings)

#### 3.2 Carry out the ridge regression!

Now we get the design matrix for the training and validation sets. 

In [ ]:
designm, ratings = getmats(traindf)
validatedm, validaterats = getmats(validatedf)

Set up a bunch of $\alpha$s `[0.01, 0.1, 10, 100, 1000]` for our ridge regression, and carry it out. Notice that we are doing just one validation set here, rather than cross-validation, in the interests of saving on running time in this homework problem. (The Learning a Model lab might help you in your code here). 

Save in a dictionary `vdict`(validation-dict) keyed by the values of $\alpha$ above the root-mean-square-error(rmse) on the predictions, and in dictionary `rdict`(regression-dict) with the same keys the sklearn ridge regression instances for each $\alpha$.

In [ ]:
#your code here
from sklearn.metrics import mean_squared_error

alphas = [0.01, 0.1, 1.0, 10, 100, 1000]
vdict={}
rdict={}

for i, alpha in enumerate(alphas):
    est = Ridge(alpha=alpha)
    est.fit(designm, ratings)
    rdict[alpha] = est
    vdict[alpha] = get_rmse(validaterats, est.predict(validatedm))
    
minalp=min(vdict, key=vdict.get)
minalp
fitmodel = rdict[minalp]
Xtrain = designm
ytrain = ratings

Lets find the $\alpha$ that minimizes the rmse's in vdict, and save in `regr` the corresponding regression

In [ ]:
alphawechoose = minalp
clf = Ridge(alpha=alphawechoose).fit(Xtrain,ytrain)

In [ ]:
minerroralpha=min(vdict, key=vdict.get)
print minerroralpha
regr=rdict[minerroralpha] 

And see our intercept and how it compares to $\bar Y$.

In [ ]:
regr.intercept_, ybar

We save our intercept and coefficients into a training dictionary similar to the `train_avg` we saved earlier.

In [ ]:
meancept=regr.intercept_
fitratmap_u={k:regr.coef_[i] for i,k in enumerate(uuids)}
fitratmap_i={k:regr.coef_[i+len(uuids)] for i,k in enumerate(uiids)}

In [ ]:
train_fits={'mean':meancept, 'users':fitratmap_u, 'items':fitratmap_i}

### Prediction!

#### 3.3 Use this model to predict on the test set.

We set up the test design matrix and get the test ratings:

In [ ]:
testdm, testrats = getmats(testdf)

Predict the ratings on the test set and store the results in `predictions['baseline_r']`. Use these to make a diagram of our results using the `compare_results` function we defined and used earlier. Also predict the ratings on the validation set and save in `predictions_valid['baseline_r']`

In [ ]:
#your code here
predictions["baseline_r"]=clf.predict(testdm)
predictions_valid["baseline_r"]=clf.predict(validatedm)
compare_results(testrats,predictions['baseline_r'], model="baseline_r", predicteds=True)

#### 3.4 Compare the results from the simple baseline model to the regularized baseline model. 

Plot both of them on the same graph using `compare_results`. HINT: use the `onsame` parameter in that function. Explain the results, both in terms of the width of the 1-standard deviation bands, and how the predictions are different from the 45 degree line.

In [ ]:
#your code here
ax,rmse=compare_results(testrats,predictions['baseline_r'], model="baseline_r", predicteds=True)
compare_results(testdf.stars,predictions['baseline'], model="baseline", onsame=ax, predicteds=True);

###### Compare
The RMSE for the baseline with regression model was better than the baseline. The regularization, as expected, created a flatter prediction line (with respect to the actuals). The baseline with regression also predicted within a tighter band of variance.

## Q4: Collaborative Filtering and The Latent Factor Model

### Model Overview

The central dogma in constructing a recommendation system using collaborative filtering is that *similar users will rate similar restaurants similarly*. Earlier we explicitly encoded this idea by using a similarity function to identify similar restaurants. We also assumed that either all users were the same (the global, maximal pooling, approach that we didnt do in this homework) or that only the current user was similar enough to make a recommendation (the user-specific approach that we did). In this section, we will use a model that allows us to identify both similar users and similar restaurants as a function of **latent factors**.

We can think of latent factors as properties of restaurants (e.g., spiciness of food or price) that users have a positive or negative preference for. We do not observe these factors or the users' preferences directly, but we assume that they affect how users tend to rate restaurants. For example, if a restaurant serves a lot of spicy food and a user dislikes spicy food, then the restaurant would have a high "spiciness" factor, and the user would have a strongly negative preference, resulting in a prediction of a low rating. Note that if users have similar preferences, then according to the model, they will behave similarly, and likewise, if restaurants have similar latent factors, they will be rated similarly by similar users. Latent factors thus give us an intuitive way to specify a generative model the obeys the central dogma, while retaining a certain explanatory globality, rather than simply a notion that one user is like another.

One issue that comes up with latent factor models is determining how many latent factors to include. There may be a number of different unmeasured properties that affect ratings in different ways -- for example, in addition to the spiciness factor above, there may also be a price factor that affects how users rate a restaurant. We can deal with the problem of choosing the number of latent factors to include in the same way we deal with choosing $K$ in a $K$-nearest neighbors problem.

### Latent Factors by Matrix Factorization

In HW4 for cs109 2013, a latent factor model was implemented using the MCMC technique Gibbs Sampling (if you take AM207 you will learn much more about this). Here we shall use another technique that builds on our regularized baseline model. This technique is called matrix factorization.

As you probably know by now, the usual method to do matrix factorization is the Singular Value Decomposition, or the SVD. However, conventional SVD is undefined when there are missing values in the matrix..in our case, the user-item matrix which has many, many missing values. 

So, how do we deal with this?

Let $r_{um}$ denote the "residual" rating of user $u$ for item $m$. By residual rating we mean whats left over after fitting the baseline, so:

$$r_{um} = Y_{um} - Y^{baseline}_{um}$$.

Let us associate with each item $m$ a vector $\bar q_{m}$ of length $L$, where L is the number of latent factors in the model. Similarly, lets associate with each user a vector $\bar p_{u}$ of length $L$. $\bar q_{m}$, either negative or positive, measures the extent to which an item(restaurant) possesses the latent factors: Low price, spicyness, etc. For the user, $\bar p_{u}$ captures the extent of interest the user has in restaurants(items) that are high on the corresponding factors. Then we write as our model:

$$r_{um} = \bar q_{m}^{T} \cdot \bar p_{u}. $$

Thus we are setting our residual equal to the interaction between user $u$ and item $m$, kind of, if you like, the user's overall interest in the item's characteristics [See http://www2.research.att.com/~volinsky/papers/ieeecomputer.pdf for more details..this is also a great reference on these techniques]. This is our first model where we attempt to capture the interaction between such quantities.

This model can be illustrated by the diagram below:

![residual](residual.png)

To solve this model we dont want to do a direct SVD as most of the matrix elements are unknown. Instead we simply solve the minimization problem over all known rankings in the training set:


$$\sum_{u,m \in train} \left( r_{um} - \bar q_{m}^{T} \cdot \bar p_{u}\right)^2$$

Notice now that we have introduced a whole lot of unknowns into the problem: $L\times(M+U)$ to be precise. We are running the danger of overfitting, again! And so we do what we did the last time, by penalizing these new unknown in the same way. Thus, we instead find the unknowns that minimize:

$$\sum_{u,m \in train} \left( r_{um} - \bar q_{m}^{T} \cdot \bar p_{u}\right)^2 +\alpha \left( \|\bar q_{m}\|^{2} + \|\bar p_{u}\|^{2} \right)$$

Now, imagine you have fixed $\alpha$ the way we do in a cross-validation loop, and furthermore, let us say we have fixed the values of $\bar q_{m}$. Then the formula above once again starts looking like a Ridge regression! There are $L$ $p_{u}$s for each u, and thus a total of $L \times U$ coefficients are to be fit, and they are regularized by the second term in the formula above. Once these coefficients are fit, we can reverse the regression, and treat their values, as values of features. There are now $L \times M$ $q_{m}$ to fit. You can see now that we can continue such an alternating process starting from some initially chosen random values of either the $p$ or the $q$. We should continue such an alternating process until the values of $\bar q_{m}$ and $\bar p_{u}$ do not change much from the previous step: we say that we have then reached convergence.

So the process we are carrying out is alternating Ridge regressions until we reach convergence. It can be shown that such a process minimizes the risk. We shall not prove this here: regardless, it ought to be kind of obvious from the structure of the algorithm. This algorithm is called **Alternating Least Squares**(ALS) as it minimizes the square error first holding the $q$ constant as features and then using the $p$s fit as constant and finding the $q$s. This minimization problem can also be solved by Stochastic Gradient descent but that is beyond the scope of this class. ALS also has the advantage that it is massively parallelizable over either the users or the items.


At this point we have only solved for the residual rating. What if we want to solve for the entire rating? We know from earlier that the solution for the baseline rating is a ridge regression which minimizes the risk

$$\sum_{u,m} \left (Y^{baseline}_{um} -  \mu - \bar \theta \cdot I_{u} - \bar \gamma \cdot I_{m} \right)^2 + \alpha \left( \theta_{u}^2 + \gamma_{m}^2 \right)$$

and gives us

$$  Y_{um}^{baseline} =  \mu +  \bar \theta \cdot I_{u} +  \bar \gamma \cdot I_{m} $$



We want to combine  $Y_{um}^{baseline}$ and $r_{um}$, and then we get:

$$Y_{um} = Y_{um}^{baseline} + r_{um},$$

which gives us

$$Y_{um} = \mu +  \bar \theta \cdot I_{u} +  \bar \gamma \cdot I_{m} + \bar q_{m}^{T} \cdot \bar p_{u}$$.

To solve this we need to simply minimize the risk of the entire regression, ie

$$\sum_{u,m} \left( Y_{um} -  \mu - \bar \theta \cdot I_{u} -  \bar \gamma \cdot I_{m} - \bar q_{m}^{T} \cdot \bar p_{u} \right)^2 + \alpha \left( \theta_{u}^2 + \gamma_{m}^2 + \|\bar q_{m}\|^{2} + \|\bar p_{u}\|^{2} \right)$$

This is just a bigger ridge regression with more unknowns! So let us go about constructing a design matrix to represent this regression, just like we did before!

#### Constructing the design matrix

The first observation to make, to practically solve this larger problem, is that we need to start with, at the very least, the same design matrix that we had earlier.

The second thing to notice is that for each user, we have $L$ factors for a total of $U \times L$ quantities. Thus, for the regression where we are fitting for the q's, or the coefficient corresponding to the item, our design matrix, which we shall call `designp`, must have $U + M + L \times U$ features. What are these features? The first $U + M$ of these features are simply the concatenation of two Indicator vectors. 

Lets assume we are talking about the user with index 3 and item with index 5, and lets use L=2. Then, the first indicator vector has a 1 at the third (index 2) position and the second indicator has a 1 at the 5th (index 4) position. Further, we go to the 5th and 6th slots in the remaining $L \times U$ features, and fill in the corresponding $p$'s $p_{51}$, and $p_{52}$. The number of coefficients we have, then are $U + M + L \times U$ $q$'s. This is shown in the top part of the diagram below.

![bigdesign](bigdesign.png)

Once we have fit for the $q$, we then use them to construct a new design matrix for the alternate regression, `designq`, in which we fit for the $p$. Now have $U + M + L \times M$ features. The first $U + M$  features are again the concatenation of two Indicator vectors, and the remaining $L \times M$ features are from the just fit $q$s. The number of coefficients we have are $U + M + L \times M$ $p$'s. This is shown in the bottom part of the diagram above.

Lets define a function `design_p` to calculate the design matrix for the first regression. Pay particular attention to the comments. Here `lshape` is the number of factors $L$, and inps is the input $p$ "coefficient" vector that we use to fill the features from the previous regression.

In [ ]:
#given the initial data frame, and the number of latent factors lshape
def design_p(indf, lshape, inps):
    #the number of columns in the added part of the feature matrix
    qshape=lshape*len(uuids)
    #the number of columns we are coming in with from the previous regression
    pshape=lshape*len(uiids)
    #the number of features from the baseline regression
    fshape=len(uuids)+len(uiids)
    #userid and itemid along with star rating from the input dataframe
    stvals=indf[['user_id', 'business_id', 'stars']].values
    #the design matrix of size N rows X M+U+L*U columns
    designp=np.zeros((stvals.shape[0], fshape+qshape))
    #ratings column vector of N rows
    ratings=np.zeros(stvals.shape[0])
    #for each row in the dataframe:
    for i, row in enumerate(stvals):
        #get userid, restaurant id and rating from the row
        userid=row[0]
        bizid=row[1]
        rat=row[2]
        #use the index corresponding to the userid and L to figure how many slots
        #in the design matrix to take up and where to take them up from
        #for e.g., if index is 2(ie third index) and L=2,this will be from index 4 on
        #(which is the 5th index. )
        posq=uuidmap[userid]*lshape
        putinat=fshape+posq

        #use the index corresponding to the business id to get the indexes of the incoming p
        posp=uiidmap[bizid]*lshape
        #fill the baseline part of the design matrix in for this row
        designp[i,:-qshape]=np.concatenate([1*(userid==uuids), 1*(bizid==uiids)])
        #set the ith element of the rating vector to the rating from the matching row
        ratings[i]=rat
        #Fill L of the slots from putinat onwards to L elements from the p matrix
        #if the index is 4(the fifth index) and L=2, this is the 8th and 9th element of the
        #p coefficients
        designp[i,putinat:putinat+lshape]=inps[posp:posp+lshape]
    #return the constructed design matrix and ratings
    return designp, ratings

#### 4.1 Write a function to compute `designq`.

In analogy with the function above, and with analogous signature, write a function `design_q` which takes as arguments the training dataframe `indf`, the number of factors `lshape`, and the input `inqs` coefficient vector of $q$s, and returns both the design matrix and the vector of ratings.

In [ ]:
#your code here
#given the initial data frame, and the number of latent factors lshape
def design_q(indf, lshape, inqs):
    #the number of columns in the added part of the feature matrix
    pshape=lshape*len(uuids)
    #the number of columns we are coming in with from the previous regression
    qshape=lshape*len(uiids)
    #the number of features from the baseline regression
    fshape=len(uuids)+len(uiids)
    #userid and itemid along with star rating from the input dataframe
    stvals=indf[['user_id', 'business_id', 'stars']].values
    #the design matrix of size N rows X M+U+L*U columns
    designq=np.zeros((stvals.shape[0], fshape+pshape))
    #ratings column vector of N rows
    ratings=np.zeros(stvals.shape[0])
    #for each row in the dataframe:
    for i, row in enumerate(stvals):
        #get userid, restaurant id and rating from the row
        userid=row[0]
        bizid=row[1]
        rat=row[2]
        #use the index corresponding to the userid and L to figure how many slots
        #in the design matrix to take up and where to take them up from
        #for e.g., if index is 2(ie third index) and L=2,this will be from index 4 on
        #(which is the 5th index. )
        posp=uuidmap[userid]*lshape
        putinat=fshape+posp

        #use the index corresponding to the business id to get the indexes of the incoming p
        posq=uiidmap[bizid]*lshape
        #fill the baseline part of the design matrix in for this row
        designq[i,:-pshape]=np.concatenate([1*(userid==uuids), 1*(bizid==uiids)])
        #set the ith element of the rating vector to the rating from the matching row
        ratings[i]=rat
        #Fill L of the slots from putinat onwards to L elements from the p matrix
        #if the index is 4(the fifth index) and L=2, this is the 8th and 9th element of the
        #p coefficients
        designq[i,putinat:putinat+lshape]=inqs[posq:posq+lshape]
    #return the constructed design matrix and ratings
    return designq, ratings

### WARNING: this part might take some hours to a day to run!

Let us set the number of factors $L$. To get your code working set L to 2, and fit for just one alpha (say $\alpha = 10$). Once you have your code working, set L to 4 and $\alpha$ to a small grid in the vicinity of what worked for the simple regularized regression. Each regression run at a L of 4 could take longer than an hour.  **So leave a lot of time for this part**. 

If you have the time, experiment with higher values of L, seeing if your rmse decreases.

In [ ]:
L=2

In [ ]:
print L*len(uiids), L*len(uuids)

Lets randomly initialize the initial set of p's and q's:

In [ ]:
initps=np.random.rand(L*len(uiids))
initqs=np.random.rand(L*len(uuids))

Now we write the main loop of our ALS code. We setup a set of $\alpha$s we will scan to find the lowest cost on the validation set. For each $\alpha$, we set the input p and q to the random initialization. We compute the design matrix `designp` using the training data frame `traindf`. We then carry out a ridge regression using the ratings and the `designp` design matrix. We save out the last $L \times U$ coefficients of this regression, and use them to supply an `inqs` to construct a `designq` matrix for the next regression. We carry out the next regression and analogously save the fit p in `inps`.

Before completing this iteration of the loop we calculate the root-mean-square error(rmse) between this iteration and the previous one (in this case the random initialization), on both `inps` and `inqs`. We save these numbers in `conv` which we use to check if our ALS has converged. We now move to the next iteration, and the next, until the rmse for both`inps` and `inqs` between the current and previous iterations has gone down to 0.005. When this happens we break out of the loop.

Since `inps` is the last thing done in the loop, we repeat the ridge regression one last time to obtain final values in `inqs`. We save this regression model in `rdict2`, along with its validation rmse for predicting on `validatedf` in `vdict`. The model predicts on the validation set using the code:

```
valdesignp, validaterats = design_p(validatedf, L, inps)
vpreds=regrp.predict(valdesignp)
rmse=get_rmse(validaterats, vpreds)
```

which constructs a `valdesignp` design matrix from the validation data frame and then uses the saved regression to predict on this design matrix.

In [ ]:
%%time
#NOTICE THE TWO ALPHAS BELOW AND READ THE EXPLANATION ABOVE
alphas=[5, 8, 10, 12, 15]
alphas=[10]
vdict2={}
rdict2={}
convdict={}
maxiters=100
for a in alphas:
    print "alpha", a
    inps=initps
    inqs=initqs
    sums=[]
    conv=[]
    reachedit=0
    for it in range(maxiters):
        #create design_p with inps randomly chosen
        designp, rats=design_p(traindf, L, inps)
        #fit
        regrp=Ridge(alpha=a).fit(designp, rats)
        inqsold=inqs
        inqs=regrp.coef_[-inqs.shape[0]:]
        #use regression coefficients as the new inqs
        designq, rats=design_q(traindf, L, inqs)
        regrq=Ridge(alpha=a).fit(designq, rats)
        inpsold=inps
        inps=regrq.coef_[-inps.shape[0]:]
        #just to see how far from 0 these are
        sums.append((inqs.sum(), inps.sum()))
        #see if the coefficients are converging
        pconv=get_rmse(inpsold, inps)
        qconv=get_rmse(inqsold, inqs)
        conv.append((pconv, qconv))
        if it > 9 and it % 10 ==0:
            print "Iteration ",it, pconv, qconv
        reachedit=it
        if pconv < 0.005 and qconv < 0.005:
            break
    #fit once more using the new inps
    designp, rats=design_p(traindf, L, inps)
    regrp=Ridge(alpha=a).fit(designp, rats)
    #Now predict on the validation set
    valdesignp, validaterats = design_p(validatedf, L, inps)
    vpreds=regrp.predict(valdesignp)
    rmse=get_rmse(validaterats, vpreds)
    vdict2[a]=rmse
    rdict2[a]=regrp
    convdict[a]=(reachedit, conv, sums)

In [ ]:
vdict2

We find the $\alpha$ which minimizes the validation set error. We look into `rdict` to find the corresponding `sklearn` regression instance, and extract the final q's from it. We construct a last `designq` using these q's and use it to fit the ridge regression one last time. 

In [ ]:
alphamin=min(vdict2, key=vdict2.get)
#Get the regression corresponding to the appropriate alpha
regrp=rdict2[alphamin]
#Extract the final q's at convergence
finalqs=regrp.coef_[-initqs.shape[0]:]
#having got final q's, use them to fit once more to get the finalps
designq, rats=design_q(traindf, L, finalqs)
regrq=Ridge(alpha=alphamin).fit(designq, rats)
#use the fit regression now to predict on the test set

#### 4.2 Make a prediction on the test set and plot your predictions compared to the basdeline model. Comment.

Use this regression `regrq` and the final q's `finalqs` to predict on the test set, using the final q's to construct the test set design matrix. Save the predictions in `predictions['svd']`.  Also save predictions on the validation set in `predictions_valid['svd']`. Use `compare_results` to plot the results on the test set and to compare to the baseline model with regularization. Comment on this comparison.

In [ ]:
#your code here
designqt, ratst=design_q(testdf, L, finalqs)
predictions["svd"]=regrq.predict(designqt)
designqv, ratsv=design_q(validatedf, L, finalqs)
predictions_valid["svd"]=regrq.predict(designqv)
ax,rmse=compare_results(ratst,predictions['svd'], model="svd", predicteds=True)
compare_results(testdf.stars,predictions['baseline_r'], model="baseline_r", onsame=ax, predicteds=True);

###### Compare
The SVD model appears to be virtually identical to the baseline with regression model. I must have done something wrong.

We plot the rate at which our regressions converge, seeing that we could perhaps have gotten away with a less stringent convergence criterion.

In [ ]:
plt.plot(convdict[alphamin][1])

## Q5 Combining results into an ensemble

From the Netflix Progress Prize 2007 paper (see http://brettb.net/project/papers/2007%20The%20BellKor%20solution%20to%20the%20Netflix%20prize.pdf):

>Predictive accuracy is substantially improved when blending multiple predictors. Our experience is that most efforts should be concentrated in deriving substantially different approaches, rather than refining a single technique. Consequently, our solution is an ensemble of many methods.
We approach blending as a linear regression problem. We ought to regress a target ratings vector on multiple predictors. The target rating vector can be the true ratings of the Probe set, and the predictors are the respective estimates for the Probe set by an ensemble of methods. The solution is the coefficients, or the weights, that should be given to each of the predictors in the ensemble.


Finally, we shall combine the ratings obtained from the kNN and the ALS approaches.

### Adding a regularized baseline to kNN

To do a really good job with this one ought to have fit the knn with coefficients from scratch. But we will use a regularized baseline for the kNN instead here, and combine the two and see what we get. 

Furthermore we should have a larger validation set to this combination, as we will combine model predictions on the validation set. This is not true here either. Thus this part of the homework serves more as illustration, and more precisely as an illustration of a new ensembling technique, **stacked** regression.

We'll approach this combination in a simple way, by carrying out a **stacked** linear regress on the *predictions* of the two models. We'll do this regression on the validation set predictions, and check our predictions on the test set.

So we'll add one more model here, `knn_r`, which adds knn on the validation and test sets onto a baseline that is created from the regularized regression `baseline_r`.

#### 5.1 Find the best fit `knn_r model` on the validation set and test sets

Analogous to what you did in 2.5 and 2.6, find the best fit model when the baseline you choose is that from the regularized regression from question 3. Hint: Use `train_fits` for the base line instead of `train_avg` in `get_ratings_user_nbd`).

In [ ]:
#your code here
#2.5
fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(12,12), tight_layout=True)

pt, at = get_ratings_user_nbd(traindf, traindf, train_fits, db, k=2, reg=1.)
compare_results(at,pt, model="knn(user) on training k=2, reg=1", predicteds=True, axis=axes[0][0])
pt, at = get_ratings_user_nbd(traindf, traindf, train_fits, db, k=2, reg=15.)
compare_results(at,pt, model="knn(user) on training k=2, reg=15", predicteds=True, axis=axes[0][1])
pt, at = get_ratings_user_nbd(traindf, traindf, train_fits, db, k=15, reg=1.)
compare_results(at,pt, model="knn(user) on training k=15, reg=1", predicteds=True, axis=axes[1][0])
pt, at = get_ratings_user_nbd(traindf, traindf, train_fits, db, k=15, reg=15.)
compare_results(at,pt, model="knn(user) on training k=15, reg=15", predicteds=True, axis=axes[1][1])

fig.show()

In [ ]:
#2.6
rmsedict2={}
for k in np.arange(1,100,25):
    for reg in np.arange(1.,100.,25.):
        pt, at = get_ratings_user_nbd(traindf, traindf, train_fits, db, k, reg)
        rmsedict2[(k,reg)]=get_rmse(at, pt)

In [ ]:
for k in np.arange(6,25,5):
    for reg in np.arange(6.,25.,5.):
        pt, at = get_ratings_user_nbd(traindf, traindf, train_fits, db, k, reg)
        rmsedict2[(k,reg)]=get_rmse(at, pt)

In [ ]:
for k in np.arange(2,11,1):
    for reg in np.arange(80.,101.,5.):
        pt, at = get_ratings_user_nbd(traindf, traindf, train_fits, db, k, reg)
        rmsedict2[(k,reg)]=get_rmse(at, pt)

In [ ]:
for reg in np.arange(81.,90.,1.):
    pt, at = get_ratings_user_nbd(traindf, traindf, train_fits, db, 3, reg)
    rmsedict2[(3,reg)]=get_rmse(at, pt)

We find the minimum.

In [ ]:
mintup2=min(rmsedict2, key=rmsedict2.get)
mintup2

In [ ]:
predictions['knn_r'], atest = get_ratings_user_nbd(testdf, traindf, train_fits, db, k=mintup2[0], reg=mintup2[1])
predictions_valid['knn_r'], avals = get_ratings_user_nbd(validatedf, traindf, train_fits, db, k=mintup2[0], reg=mintup2[1])

In [ ]:
compare_results(atest,predictions['knn_r'], model="knn(user) on test k=3, reg=84", predicteds=True)

What models have we collected so far?

In [ ]:
predictions.keys()

In [ ]:
print get_rmse(atest, predictions['knn'])
print get_rmse(atest, predictions['baseline_r'])
print get_rmse(atest, predictions['svd'])
print get_rmse(atest, predictions['baseline'])
print get_rmse(atest, predictions['knn_r'])

We construct a dataframe with the two predictions and the actual value on the **Validation** set.

In [ ]:
dfensemble=pd.DataFrame.from_dict({'knn_r':predictions_valid['knn_r'],
                                   'svd':predictions_valid['svd'], 
                                   'baseline_r':predictions_valid['baseline_r'], 'y':validatedf.stars.values})

#### 5.2 Carry out an unregularized linear regression of the actual values against the three predictions

Use the `y` values obtained from the validation set in the dataframe `dfensemble`. Regress against the other columns from the validation set. Such a regression is called 'stacking'. Store the regression model in the variable `valreg`

In [ ]:
#your code here
from sklearn.linear_model import LinearRegression
X = dfensemble.drop('y', axis = 1)
lm = LinearRegression()
valreg = lm.fit(X, dfensemble.y)

#### 5.3 Get the same models on the test set and use linear regression to calculate the predictions. Comment

 Store the predictions on the test set in the dataframe `dfensembletest` which should have columns exactly analogous to those on `dfensemble`. Use this dataframe in the prediction process on the test set. Store in the variable `epreds` the predictions of our stacked regression `valreg`, and the actual values of the ratings from the test set in the variable `testactual`. Comment on the results, using the diagram we plot below and the value of the RMSE as compared to before.

In [ ]:
#your code here
dfensembletest=pd.DataFrame.from_dict({'knn_r':predictions['knn_r'],
                                       'svd':predictions['svd'], 
                                       'baseline_r':predictions['baseline_r'], 'y':testdf.stars.values})

Xt = dfensembletest.drop('y', axis = 1)
epreds = valreg.predict(Xt)
testactual = testdf.stars.values

Lets plot the results and see the RMSE:

In [ ]:
ax=compare_results(testactual, epreds, model="ensemble", predicteds=True);

###### Comments
While the ensemble RMSE appears to be lower than all of the individual models, I'm a little disappointed that it wasn't significantly lower. The band showing variance appears to be narrower than most of the other models.

See http://web.stanford.edu/~lmackey/papers/netflix_story-nas11-slides.pdf for a fun presentation on Ensembling and the Netflix Prize. If you'd like to learn more, please read Chris Volinksy's papers on the Netflix prize. There are also comprehensive reviews [here](http://arxiv.org/abs/1202.1112) and [here](http://www.grouplens.org/system/files/FnT%20CF%20Recsys%20Survey.pdf).

FIN